This notebook was used to produce Table 2 of
>Anton Osokin, Anatole Chessel, Rafael E. Carazo Salas, Federico Vaggi<br>
GANs for Biological Image Synthesis<br>
In ICCV, 2017

In [1]:
import os
import re
import numpy as np
import copy

In [2]:
# parameters of the experiment
model_iter_of_interest = 50000
num_diter = 5000

log_path = 'logs_eval_6class_conditional_fake_vs_real'
model_names = ['size-48-80_6class_wgangp-independent-adam',
               'size-48-80_6class_wgangp-independent-sep-adam',
               'size-48-80_6class_wgangp-multichannel-adam',
               'size-48-80_6class_wgangp-multichannel-sep-adam',
               'size-48-80_6class_wgangp-star-shaped-adam']
model_names_for_legend = ['WGAN-GP independent',
                          'WGAN-GP independent-separable',
                          'WGAN-GP multi-channel',
                          'WGAN-GP multi-channel separable',
                          'WGAN-GP star-shaped']
eval_methods = ['WGAN-GP'] #['GAN', 'WGAN', 'WGAN-GP']

file_template = 'seed{0}_split{1}_{2}_trainIter{3}_eval{4}_iter{5}.txt'

class_names = ['Alp14', 'Arp3', 'Cki2', 'Mkh1', 'Sid2', 'Tea1']

num_splits = 10
rand_seed = 0

In [3]:
def read_last_lines(file_name):
    with open(file_name) as f:
        lines = list(f)
        return lines[-1], lines[-2]
    
def get_joint_score(file_name):
    score = float('nan')
    try:
        last_line, _ = read_last_lines(file_name)
        numeric_const_pattern = "[-+]?(?:(?:\d*\.\d+)|(?:\d+\.?))(?:[Ee][+-]?\d+)?"
        score_pattern = 'equals\s*(%s)' % numeric_const_pattern
        match = re.search(score_pattern, last_line)
        if match:
            score = float(match.group(1))
    except:
        print('Could not extract value from {0}'.format(file_name))
    return score

def get_scores(file_name):
    score = float('nan')
    class_scores = {}
    try:
        last_line, second_last_line = read_last_lines(file_name)
        numeric_const_pattern = "[-+]?(?:(?:\d*\.\d+)|(?:\d+\.?))(?:[Ee][+-]?\d+)?"
        score_pattern = 'equals\s*(%s)' % numeric_const_pattern
        match = re.search(score_pattern, last_line)
        if match:
            score = float(match.group(1))
            
        class_scores = {}
        for cl_name in class_names:
            score_pattern = '%s:\s*(%s);' % (cl_name, numeric_const_pattern)
            match = re.search(score_pattern, second_last_line)
            if match:
                cl_score = float(match.group(1))
            else:
                cl_score = float('nan')
            class_scores[cl_name] = cl_score
    except:
        print('Could not extract value from {0}'.format(file_name))
    return score, class_scores

In [4]:
mean_all = np.zeros((len(eval_methods), len(class_names) + 1, len(model_names)))
std_all = np.zeros((len(eval_methods), len(class_names) + 1, len(model_names)))
for i_eval, eval_method in enumerate(eval_methods):
    for i_model, model_name in enumerate(model_names):
        split_scores, split_class_scores = [], []
        for i_split in range(num_splits):
            file_name = os.path.join(log_path, file_template)
            file_name = file_name.format(rand_seed, i_split, model_name, model_iter_of_interest, eval_method, num_diter)
            score, class_scores = get_scores(file_name)
            split_scores.append(score)
            split_class_scores.append(class_scores)
        
    
        joint_data = np.array(split_scores)
        mean_all[i_eval, -1, i_model] = np.nanmean(joint_data, axis=0)
        std_all[i_eval, -1, i_model] = np.nanstd(joint_data, axis=0)
            
        for i_cl, class_name in enumerate(class_names):
            class_data = np.array([s[class_name] for s in split_class_scores])
            mean_all[i_eval, i_cl, i_model] = np.nanmean(class_data, axis=0)
            std_all[i_eval, i_cl, i_model] = np.nanstd(class_data, axis=0)

In [5]:
def do_table_latex(mean, std, class_names):
    n_cl = len(class_names)
    for m in model_names_for_legend:
        print(' & {0}'.format(m), end=' ')
    print('\\\\')
    cl_names = copy.deepcopy(class_names)
    cl_names.append('6 proteins')
    for i in range(n_cl + 1):
        print('\\protein{{{0}}}'.format(cl_names[i]), end=' ')
        gray_col = np.argmin(mean[i])
        for j in range(len(model_names_for_legend)):
            print('& {0:0.2f} $\pm$ {1:0.2f}'.format(mean[i, j], std[i, j]),end=' ')
                    
        print('\\\\')
        
        
def do_table(mean, std, class_names):
    n_cl = len(class_names)
    for m in model_names_for_legend:
        print('\t{0}'.format(m), end=' ')
    print('')
    cl_names = copy.deepcopy(class_names)
    cl_names.append('all')
    for i in range(n_cl + 1):
        print('{0}'.format(cl_names[i]), end=' ')
        for j in range(len(model_names_for_legend)):
            print('\t {0:0.2f}\u00B1{1:0.2f}'.format(mean[i, j], std[i, j]),end=' ')
        print('')

In [6]:
do_latex = False
for i_eval in range(len(eval_methods)):
    print('Evaluation with', eval_methods[i_eval])
    if do_latex:
        do_table_latex(mean_all[i_eval], std_all[i_eval], class_names)
    else:
        do_table(mean_all[i_eval], std_all[i_eval], class_names)
    print('')

Evaluation with WGAN-GP
	WGAN-GP independent 	WGAN-GP independent-separable 	WGAN-GP multi-channel 	WGAN-GP multi-channel separable 	WGAN-GP star-shaped 
Alp14 	 0.56±0.27 	 1.24±0.15 	 3.16±0.36 	 2.35±0.52 	 0.64±0.34 
Arp3 	 1.23±0.26 	 2.36±0.37 	 3.20±0.36 	 4.23±0.36 	 2.10±0.47 
Cki2 	 0.35±0.52 	 0.97±0.35 	 2.49±0.31 	 3.64±0.52 	 1.19±0.29 
Mkh1 	 0.78±0.55 	 0.53±0.41 	 4.58±0.47 	 6.64±0.46 	 2.40±0.56 
Sid2 	 0.84±0.40 	 1.03±0.50 	 4.49±0.51 	 3.21±0.57 	 1.08±0.56 
Tea1 	 0.83±0.51 	 0.76±0.47 	 4.38±0.32 	 2.84±0.47 	 1.07±0.37 
all 	 0.76±0.16 	 1.15±0.16 	 3.72±0.14 	 3.82±0.25 	 1.41±0.15 



In [7]:
# getting real vs real numbers
log_path_real_vs_real = '../real_vs_real/logs_eval_8class_real_vs_real'
eval_methods = ['WGAN-GP']

file_template_real_vs_real = 'seed{0}_split{1}_eval{2}_iter{3}_classes{4}{5}.txt'

In [8]:
scores_real_vs_real = np.zeros((len(eval_methods), len(class_names), num_splits))
for i_eval, eval_method in enumerate(eval_methods):
    print('Evaluation with', eval_methods[i_eval])
    for i_cl, cl_name in enumerate(class_names):
        split_data = np.zeros(num_splits)
        for i_split in range(num_splits):
            file_name = os.path.join(log_path_real_vs_real, file_template_real_vs_real)
            file_name = file_name.format(rand_seed, i_split, eval_method, num_diter, cl_name, cl_name)
            scores_real_vs_real[i_eval, i_cl, i_split] = get_joint_score(file_name)
        print('{0}: {1:0.1f} \u00B1 {2:0.1f}'.format(cl_name,
                                                     np.mean(scores_real_vs_real[i_eval, i_cl]),
                                                     np.std(scores_real_vs_real[i_eval, i_cl])))
    joint_scores = np.mean(scores_real_vs_real[i_eval], axis=0)
    print('Together: {0:0.1f} \u00B1 {1:0.1f}'.format(np.mean(joint_scores), np.std(joint_scores)))

Evaluation with WGAN-GP
Alp14: 0.1 ± 0.2
Arp3: 0.8 ± 0.4
Cki2: -0.2 ± 0.3
Mkh1: -0.2 ± 0.4
Sid2: -0.6 ± 0.3
Tea1: -0.1 ± 0.4
Together: -0.1 ± 0.2
